In [1]:
import numpy as np
from pyriemann.utils.base import logm, expm
from scipy.linalg import sqrtm

# Classe CalculCovariance
class CalculCovariance:
    def __init__(self, p_points, stride):
        """
        Initialise une nouvelle instance de la classe CalculCovariance.
        
        :param p_points: Nombre de points de données pour chaque calcul de matrice de covariance.
        :param stride: Intervalle de décalage entre chaque calcul de matrice de covariance.
        """
        self.p_points = p_points  # Nombre de points de données pour chaque calcul de matrice de covariance
        self.stride = stride  # Intervalle de décalage entre chaque calcul de matrice de covariance
    
    def calculer_matrices_covariance(self, donnees):
        """
        Calcule les matrices de covariance pour les données fournies.
        
        :param donnees: Un numpy array où chaque ligne est une information et chaque colonne un point de données.
        :return: Liste des matrices de covariance de taille KxK, où K est le nombre de lignes dans les données.
        """
        k, n = donnees.shape  # k = nombre d'informations, n = nombre de points de données
        matrices_covariance = []
        
        for start_index in range(0, n - self.p_points + 1, self.stride):
            end_index = start_index + self.p_points
            segment = donnees[:, start_index:end_index]
            # Calculer la covariance entre les lignes (informations)
            covariance_matrix = np.cov(segment)
            matrices_covariance.append(covariance_matrix)
        
        return matrices_covariance

# Classe Trajectoire
class Trajectoire:
    def __init__(self, *covariance_matrices):
        self.covariance_matrices = list(covariance_matrices)
        self.trajectoire = self.calculer_trajectoire_initiale()
    
    def ajouter_covariance(self, nouvelle_covariance):
        self.covariance_matrices.append(nouvelle_covariance)
        self.trajectoire = self.calculer_trajectoire_apres_ajout()
    
    def calculer_trajectoire_initiale(self):
        trajectoire = np.zeros_like(self.covariance_matrices[0])
        for cov in self.covariance_matrices:
            trajectoire += cov
        return trajectoire
    
    def calculer_trajectoire_apres_ajout(self):
        trajectoire = np.zeros_like(self.covariance_matrices[0])
        for cov in self.covariance_matrices:
            trajectoire += cov
        return trajectoire
    
    def tangent_space_projection(self, A, B):
        
        # Calcul de la racine carrée de B
        sqrt_B = sqrtm(B)

        # Calcul de B^(-1/2)
        sqrt_inv_B = np.linalg.inv(sqrt_B)

        # Calcul de B^(1/2) A B^(-1/2)
        transformed_A = np.dot(np.dot(sqrt_inv_B, A), sqrt_inv_B)

        # Calcul du logarithme matriciel de B^(1/2) A B^(-1/2)
        log_transformed_A = logm(transformed_A)

        # Projet dans l'espace tangent de B
        tangent_space_A = np.dot(np.dot(sqrt_B, log_transformed_A), sqrt_B)

        return tangent_space_A

    def inverse_tangent_space_projection(self, ProjB_A, B):
        # Calcul de la racine carrée de B
        sqrt_B = sqrtm(B)
        # Calcul de B^(-1/2)
        sqrt_inv_B = np.linalg.inv(sqrt_B)
        # Calcul de B^(-1/2) * ProjB_A * B^(-1/2)
        transformed_ProjB_A = np.dot(np.dot(sqrt_inv_B, ProjB_A), sqrt_inv_B)
        # Calcul de l'exponentielle matricielle de B^(-1/2) * ProjB_A * B^(-1/2)
        exp_transformed_ProjB_A = expm(transformed_ProjB_A)
        # Revenir à l'espace original
        original_A = np.dot(np.dot(sqrt_B, exp_transformed_ProjB_A), sqrt_B)
        return original_A
    
    def transport_vector(self, Delta, B, A):
        # Calcul de l'inverse de B
        inv_B = np.linalg.inv(B)
        # Calcul de A * B^(-1)
        A_invB = np.dot(A, inv_B)
        # Calcul de E = (A * B^(-1))^(1/2)
        E = sqrtm(A_invB)
        # Transport de Delta du plan tangent T_A vers T_B en utilisant la transposée de E
        transported_Delta = np.dot(np.dot(E, Delta), E.T)
        return transported_Delta
    
    def obtenir_covariances(self):
        return self.covariance_matrices
    
    def obtenir_trajectoire(self):
        return self.trajectoire
    
    def __str__(self):
        return (f'Trajectoire avec {len(self.covariance_matrices)} matrices de covariance\n'
                f'Trajectoire actuelle:\n{self.trajectoire}')

# Classe GestionnaireTrajectoires
class GestionnaireTrajectoires:
    def __init__(self, max_matrices_par_trajectoire, *initial_covariances):
        self.trajectoires = []
        self.max_matrices_par_trajectoire = max_matrices_par_trajectoire
        if initial_covariances:
            traj = Trajectoire(*initial_covariances[:max_matrices_par_trajectoire])
            self.trajectoires.append(traj)
            remaining_covariances = initial_covariances[max_matrices_par_trajectoire:]
            while remaining_covariances:
                traj = Trajectoire(*remaining_covariances[:max_matrices_par_trajectoire])
                self.trajectoires.append(traj)
                remaining_covariances = remaining_covariances[max_matrices_par_trajectoire:]
    
    def ajouter_covariance(self, nouvelle_covariance):
        if not self.trajectoires or len(self.trajectoires[-1].obtenir_covariances()) >= self.max_matrices_par_trajectoire:
            self.trajectoires.append(Trajectoire(nouvelle_covariance))
        else:
            self.trajectoires[-1].ajouter_covariance(nouvelle_covariance)
    
    def obtenir_trajectoires(self):
        return [traj.obtenir_trajectoire() for traj in self.trajectoires]
    
    def __str__(self):
        result = ''
        for i, traj in enumerate(self.trajectoires):
            result += f'Trajectoire {i+1}:\n{traj}\n'
        return result

# Fonction de test
def test_gestionnaire_trajectoires(donnees, p_points,stride, nb_cov_matrix_per_traj, file):
    file.write("----- Test GestionnaireTrajectoires -----\n")
    file.write(f"Données:\n{donnees}\n")
    file.write(f"p_points: {p_points}\n")
    file.write(f"stride: {stride}\n")
    file.write(f"Nombre max de matrices par trajectoire: {nb_cov_matrix_per_traj}\n")

    # Création de l'instance CalculCovariance
    calcul_cov = CalculCovariance(p_points,stride)
    
    # Calcul des matrices de covariance
    matrices_covariance = calcul_cov.calculer_matrices_covariance(donnees)
    
    # Instanciation du gestionnaire de trajectoires avec un maximum de p_points matrices par trajectoire
    gestionnaire = GestionnaireTrajectoires(nb_cov_matrix_per_traj, *matrices_covariance)
    
    # Écriture de l'état initial des trajectoires dans le fichier
    file.write("État initial des trajectoires:\n")
    file.write(str(gestionnaire))
    file.write("\n")
    
    # Ajout d'une nouvelle matrice de covariance
    nouvelle_cov = np.cov(donnees[:, :p_points])
    gestionnaire.ajouter_covariance(nouvelle_cov)
    
    # Écriture de l'état des trajectoires après ajout de la nouvelle matrice dans le fichier
    file.write("État des trajectoires après ajout de la nouvelle matrice:\n")
    file.write(str(gestionnaire))
    file.write("\n----- Fin du test -----\n\n")

# Exemple de tests avec différentes tailles de matrices et différents nombres de matrices par trajectoire
if __name__ == "__main__":
    # Données prédéfinies pour les tests
    donnees1 = np.array([
        [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
        [2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
        [3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
    ])

    donnees2 = np.array([
        [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
        [15, 25, 35, 45, 55, 65, 75, 85, 95, 105],
        [20, 30, 40, 50, 60, 70, 80, 90, 100, 110],
        [25, 35, 45, 55, 65, 75, 85, 95, 105, 115]
    ])

    donnees3 = np.array([
        [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80],
        [6, 12, 18, 24, 30, 36, 42, 48, 54, 60, 66, 72, 78, 84, 90, 96],
        [7, 14, 21, 28, 35, 42, 49, 56, 63, 70, 77, 84, 91, 98, 105, 112],
        [8, 16, 24, 32, 40, 48, 56, 64, 72, 80, 88, 96, 104, 112, 120, 128],
        [9, 18, 27, 36, 45, 54, 63, 72, 81, 90, 99, 108, 117, 126, 135, 144]
    ])

    scenarios = [
        (donnees1, 3,1, 2),  # 3 informations, 10 points de données, 3 points par matrice, 2 matrices par trajectoire
        (donnees2, 5,1, 3),  # 4 informations, 10 points de données, 5 points par matrice, 3 matrices par trajectoire
        (donnees3, 6,1, 4),  # 5 informations, 16 points de données, 6 points par matrice, 4 matrices par trajectoire
    ]

    with open("resultats_tests.txt", "w") as file:
        for donnees, p_points,stride, nb_cov_matrix_per_traj in scenarios:
            test_gestionnaire_trajectoires(donnees, p_points,stride, nb_cov_matrix_per_traj, file)


In [6]:
import numpy as np
from pyriemann.utils.base import logm, expm
from scipy.linalg import sqrtm

# Classe CalculCovariance
class CalculCovariance:
    def __init__(self, p_points, stride):
        self.p_points = p_points
        self.stride = stride
    
    def calculer_matrices_covariance(self, donnees):
        k, n = donnees.shape
        matrices_covariance = []
        
        for start_index in range(0, n - self.p_points + 1, self.stride):
            end_index = start_index + self.p_points
            segment = donnees[:, start_index:end_index]
            covariance_matrix = np.cov(segment)
            matrices_covariance.append(covariance_matrix)
        
        return matrices_covariance

# Classe Trajectoire
class Trajectoire:
    def __init__(self, *covariance_matrices):
        self.covariance_matrices = list(covariance_matrices)
        self.trajectoire = None
    
    def ajouter_covariance(self, nouvelle_covariance):
        self.covariance_matrices.append(nouvelle_covariance)
        self.trajectoire = None
    
    def calculer_trajectoire(self):
        self.trajectoire = sum(self.covariance_matrices)
    
    def obtenir_covariances(self):
        return self.covariance_matrices
    
    def obtenir_trajectoire(self):
        if self.trajectoire is None:
            self.calculer_trajectoire()
        return self.trajectoire
    
    def __str__(self):
        return (f'Trajectoire avec {len(self.covariance_matrices)} matrices de covariance\n'
                f'Trajectoire actuelle:\n{self.trajectoire}')

# Classe GestionnaireTrajectoires
class GestionnaireTrajectoires:
    def __init__(self, max_matrices_par_trajectoire, *initial_covariances):
        self.trajectoires = []
        self.max_matrices_par_trajectoire = max_matrices_par_trajectoire
        if initial_covariances:
            traj = Trajectoire(*initial_covariances[:max_matrices_par_trajectoire])
            self.trajectoires.append(traj)
            remaining_covariances = initial_covariances[max_matrices_par_trajectoire:]
            while remaining_covariances:
                traj = Trajectoire(*remaining_covariances[:max_matrices_par_trajectoire])
                self.trajectoires.append(traj)
                remaining_covariances = remaining_covariances[max_matrices_par_trajectoire:]
    
    def ajouter_covariance(self, nouvelle_covariance):
        if not self.trajectoires or len(self.trajectoires[-1].obtenir_covariances()) >= self.max_matrices_par_trajectoire:
            self.trajectoires.append(Trajectoire(nouvelle_covariance))
        else:
            self.trajectoires[-1].ajouter_covariance(nouvelle_covariance)
    
    def obtenir_trajectoires(self):
        return [traj.obtenir_trajectoire() for traj in self.trajectoires]
    
    def tangent_space_projection(self, A, B):
        sqrt_B = sqrtm(B)
        sqrt_inv_B = np.linalg.inv(sqrt_B)
        transformed_A = np.dot(np.dot(sqrt_inv_B, A), sqrt_inv_B)
        log_transformed_A = logm(transformed_A)
        return log_transformed_A
    
    def inverse_tangent_space_projection(self, ProjB_A, B):
        sqrt_B = sqrtm(B)
        sqrt_inv_B = np.linalg.inv(sqrt_B)
        transformed_ProjB_A = np.dot(np.dot(sqrt_inv_B, ProjB_A), sqrt_inv_B)
        exp_transformed_ProjB_A = expm(transformed_ProjB_A)
        original_A = np.dot(np.dot(sqrt_B, exp_transformed_ProjB_A), sqrt_B)
        return original_A
    
    def transport_vector(self, Delta, A, B):
        inv_B = np.linalg.inv(B)
        A_invB = np.dot(A, inv_B)
        E = sqrtm(A_invB)
        transported_Delta = np.dot(np.dot(E, Delta), E.T)
        return transported_Delta

    def projeter_et_transporter(self):
        if len(self.trajectoires) == 0:
            return

        for traj in self.trajectoires:
            covariances = traj.obtenir_covariances()
            projected_covariances = []

            for i in range(len(covariances) - 1):
                proj = self.tangent_space_projection(covariances[i], covariances[i + 1])
                if projected_covariances:
                    for j in range(len(projected_covariances)):
                        projected_covariances[j] = self.transport_vector(projected_covariances[j], covariances[i], covariances[i + 1])
                projected_covariances.append(proj)

            traj.covariance_matrices = projected_covariances
            traj.calculer_trajectoire()
    
    def __str__(self):
        result = ''
        for i, traj in enumerate(self.trajectoires):
            result += f'Trajectoire {i+1}:\n{traj}\n'
        return result

# Fonction de test
def test_gestionnaire_trajectoires(donnees, p_points, stride, nb_cov_matrix_per_traj, file):
    file.write("----- Test GestionnaireTrajectoires -----\n")
    file.write(f"Données:\n{donnees}\n")
    file.write(f"p_points: {p_points}\n")
    file.write(f"stride: {stride}\n")
    file.write(f"Nombre max de matrices par trajectoire: {nb_cov_matrix_per_traj}\n")

    calcul_cov = CalculCovariance(p_points, stride)
    matrices_covariance = calcul_cov.calculer_matrices_covariance(donnees)
    
    gestionnaire = GestionnaireTrajectoires(nb_cov_matrix_per_traj, *matrices_covariance)
    gestionnaire.projeter_et_transporter()
    
    file.write("État initial des trajectoires:\n")
    file.write(str(gestionnaire) + "\n")
    
    nouvelle_covariance = np.cov(donnees[:, -p_points:])
    gestionnaire.ajouter_covariance(nouvelle_covariance)
    gestionnaire.projeter_et_transporter()
    
    file.write("État des trajectoires après ajout de la nouvelle covariance:\n")
    file.write(str(gestionnaire) + "\n")

if __name__ == "__main__":
    donnees1 = np.array([
        [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
        [2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
        [3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
    ])

    donnees2 = np.array([
        [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
        [15, 25, 35, 45, 55, 65, 75, 85, 95, 105],
        [20, 30, 40, 50, 60, 70, 80, 90, 100, 110],
        [25, 35, 45, 55, 65, 75, 85, 95, 105, 115]
    ])

    donnees3 = np.array([
        [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80],
        [6, 12, 18, 24, 30, 36, 42, 48, 54, 60, 66, 72, 78, 84, 90, 96],
        [7, 14, 21, 28, 35, 42, 49, 56, 63, 70, 77, 84, 91, 98, 105, 112],
        [8, 16, 24, 32, 40, 48, 56, 64, 72, 80, 88, 96, 104, 112, 120, 128],
        [9, 18, 27, 36, 45, 54, 63, 72, 81, 90, 99, 108, 117, 126, 135, 144]
    ])

    scenarios = [
        (donnees1, 3, 1, 2),
        (donnees2, 5, 1, 3),
        (donnees3, 6, 1, 4),
    ]

    with open("resultats_tests.txt", "w") as file:
        for donnees, p_points, stride, nb_cov_matrix_per_traj in scenarios:
            test_gestionnaire_trajectoires(donnees, p_points, stride, nb_cov_matrix_per_traj, file)


ValueError: Matrices must be positive definite. Add regularization to avoid this error.

In [19]:
import numpy as np
from pyriemann.utils.base import logm, expm
from scipy.linalg import sqrtm, eigh

# Classe CalculCovariance
class CalculCovariance:
    def __init__(self, p_points, stride):
        self.p_points = p_points
        self.stride = stride
    
    def calculer_matrices_covariance(self, donnees):
        k, n = donnees.shape
        matrices_covariance = []
        
        for start_index in range(0, n - self.p_points + 1, self.stride):
            end_index = start_index + self.p_points
            segment = donnees[:, start_index:end_index]
            covariance_matrix = np.cov(segment) + np.eye(k) * 1e-6  # Régularisation ajoutée
            matrices_covariance.append(covariance_matrix)
        
        return matrices_covariance
    
# Classe Trajectoire
class Trajectoire:
    def __init__(self, *covariance_matrices):
        self.covariance_matrices = list(covariance_matrices)
        self.trajectoire = None
    
    def ajouter_covariance(self, nouvelle_covariance):
        self.covariance_matrices.append(nouvelle_covariance)
        self.trajectoire = None
    
    def calculer_trajectoire(self):
        self.trajectoire = sum(self.covariance_matrices)
    
    def obtenir_covariances(self):
        return self.covariance_matrices
    
    def obtenir_trajectoire(self):
        if self.trajectoire is None:
            self.calculer_trajectoire()
        return self.trajectoire
    
    def __str__(self):
        return (f'Trajectoire avec {len(self.covariance_matrices)} matrices de covariance\n'
                f'Trajectoire actuelle:\n{self.trajectoire}')

# Classe GestionnaireTrajectoires
class GestionnaireTrajectoires:
    def __init__(self, max_matrices_par_trajectoire, *initial_covariances):
        self.trajectoires = []
        self.max_matrices_par_trajectoire = max_matrices_par_trajectoire
        if initial_covariances:
            traj = Trajectoire(*initial_covariances[:max_matrices_par_trajectoire])
            self.trajectoires.append(traj)
            remaining_covariances = initial_covariances[max_matrices_par_trajectoire:]
            while remaining_covariances:
                traj = Trajectoire(*remaining_covariances[:max_matrices_par_trajectoire])
                self.trajectoires.append(traj)
                remaining_covariances = remaining_covariances[max_matrices_par_trajectoire:]
    
    def ajouter_covariance(self, nouvelle_covariance):
        if not self.trajectoires or len(self.trajectoires[-1].obtenir_covariances()) >= self.max_matrices_par_trajectoire:
            self.trajectoires.append(Trajectoire(nouvelle_covariance))
        else:
            self.trajectoires[-1].ajouter_covariance(nouvelle_covariance)
    
    def obtenir_trajectoires(self):
        return [traj.obtenir_trajectoire() for traj in self.trajectoires]
    
    def tangent_space_projection(self, A, B):
        sqrt_B = sqrtm(B)
        sqrt_inv_B = np.linalg.inv(sqrt_B)
        transformed_A = np.dot(np.dot(sqrt_inv_B, A), sqrt_inv_B)
        log_transformed_A = logm(transformed_A)
        return log_transformed_A
    
    def inverse_tangent_space_projection(self, ProjB_A, B):
        sqrt_B = sqrtm(B)
        sqrt_inv_B = np.linalg.inv(sqrt_B)
        transformed_ProjB_A = np.dot(np.dot(sqrt_inv_B, ProjB_A), sqrt_inv_B)
        exp_transformed_ProjB_A = expm(transformed_ProjB_A)
        original_A = np.dot(np.dot(sqrt_B, exp_transformed_ProjB_A), sqrt_B)
        return original_A
    
    def transport_vector(self, Delta, A, B):
        inv_B = np.linalg.inv(B)
        A_invB = np.dot(A, inv_B)
        E = sqrtm(A_invB)
        transported_Delta = np.dot(np.dot(E, Delta), E.T)
        return transported_Delta

    def projeter_et_transporter(self):
        if len(self.trajectoires) == 0:
            return

        for traj in self.trajectoires:
            covariances = traj.obtenir_covariances()
            projected_covariances = []

            for i in range(len(covariances) - 1):
                proj = self.tangent_space_projection(covariances[i], covariances[i + 1])
                if projected_covariances:
                    for j in range(len(projected_covariances)):
                        projected_covariances[j] = self.transport_vector(projected_covariances[j], covariances[i], covariances[i + 1])
                projected_covariances.append(proj)

            # Ajouter la dernière covariance sans projection car elle est déjà au bon endroit
            projected_covariances.append(covariances[-1])

            # Remplacer les covariances dans la trajectoire par les covariances projetées et transportées
            traj.covariance_matrices = projected_covariances

            # Calculer la trajectoire après projection et transport
            traj.calculer_trajectoire()
    
    def __str__(self):
        result = ''
        for i, traj in enumerate(self.trajectoires):
            result += f'Trajectoire {i+1}:\n{traj}\n'
        return result

# Fonction de test
def test_gestionnaire_trajectoires(donnees, p_points, stride, nb_cov_matrix_per_traj, file):
    file.write("----- Test GestionnaireTrajectoires -----\n")
    file.write(f"Données:\n{donnees}\n")
    file.write(f"p_points: {p_points}\n")
    file.write(f"stride: {stride}\n")
    file.write(f"Nombre max de matrices par trajectoire: {nb_cov_matrix_per_traj}\n")

    calcul_cov = CalculCovariance(p_points, stride)
    matrices_covariance = calcul_cov.calculer_matrices_covariance(donnees)
    gestionnaire = GestionnaireTrajectoires(nb_cov_matrix_per_traj, *matrices_covariance)
    gestionnaire.projeter_et_transporter()
    file.write("État initial des trajectoires:\n")
    file.write(str(gestionnaire) + "\n")
    
    nouvelle_covariance = np.cov(donnees[:, -p_points:]) + np.eye(donnees.shape[0]) * 1e-6
    gestionnaire.ajouter_covariance(nouvelle_covariance)
    gestionnaire.projeter_et_transporter()
    
    file.write("État des trajectoires après ajout de la nouvelle covariance:\n")
    file.write(str(gestionnaire) + "\n")

def generate_less_correlated_data(num_samples, num_features):
    data = np.random.normal(size=(num_features, num_samples))
    for i in range(1, num_features):
        data[i] += data[i - 1] * 0.1  # Corrélation faible entre les fonctionnalités adjacentes
    return data

if __name__ == "__main__":
    donnees1 = np.array([
        [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
        [2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
        [3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
    ])

    donnees2 = np.array([
        [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
        [15, 25, 35, 45, 55, 65, 75, 85, 95, 105],
        [20, 30, 40, 50, 60, 70, 80, 90, 100, 110],
        [25, 35, 45, 55, 65, 75, 85, 95, 105, 115]
    ])

    donnees3 = np.array([
        [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80],
        [6, 12, 18, 24, 30, 36, 42, 48, 54, 60, 66, 72, 78, 84, 90, 96],
        [7, 14, 21, 28, 35, 42, 49, 56, 63, 70, 77, 84, 91, 98, 105, 112],
        [8, 16, 24, 32, 40, 48, 56, 64, 72, 80, 88, 96, 104, 112, 120, 128],
        [9, 18, 27, 36, 45, 54, 63, 72, 81, 90, 99, 108, 117, 126, 135, 144]
    ])

    scenarios = [
        (donnees1, 3, 1, 2),
        (donnees2, 5, 1, 3),
        (donnees3, 6, 1, 4),
    ]
    donnees1 = generate_less_correlated_data(10, 2)
    donnees2 = generate_less_correlated_data(10, 3)
    donnees3 = generate_less_correlated_data(15, 4)


    with open("resultats_tests.txt", "w") as file:
        for donnees, p_points, stride, nb_cov_matrix_per_traj in scenarios:
            test_gestionnaire_trajectoires(donnees, p_points, stride, nb_cov_matrix_per_traj, file)


In [20]:
import numpy as np

# Définir le nombre de points de données et la dimension
n_points = 1000
dimension = 3

# Générer du bruit blanc (données normales indépendantes)
data = np.random.randn(n_points, dimension)

# Définir deux matrices de covariance
cov_matrix_1 = np.array([[1, 0.8, 0.6],
                         [0.8, 1, 0.8],
                         [0.6, 0.8, 1]])

cov_matrix_2 = np.array([[1, -0.4, 0.3],
                         [-0.4, 1, -0.2],
                         [0.3, -0.2, 1]])

# Vérifier que les matrices sont définies positives
print("Covariance Matrix 1 is positive definite:", np.all(np.linalg.eigvals(cov_matrix_1) > 0))
print("Covariance Matrix 2 is positive definite:", np.all(np.linalg.eigvals(cov_matrix_2) > 0))

# Appliquer les matrices de covariance aux données
# Les matrices de covariance doivent être des matrices carrées et de la même dimension que les données
data_transformed_1 = np.dot(data[:n_points//2], np.linalg.cholesky(cov_matrix_1))
data_transformed_2 = np.dot(data[n_points//2:], np.linalg.cholesky(cov_matrix_2))

# Combiner les données transformées
data_transformed = np.vstack((data_transformed_1, data_transformed_2))

# Calculer les matrices de covariance
cov_estimated_1 = np.cov(data_transformed_1, rowvar=False)
cov_estimated_2 = np.cov(data_transformed_2, rowvar=False)

print("Estimated Covariance Matrix 1:\n", cov_estimated_1)
print("Estimated Covariance Matrix 2:\n", cov_estimated_2)

# Vérifier si les matrices de covariance estimées sont définies positives
print("Estimated Covariance Matrix 1 is positive definite:", np.all(np.linalg.eigvals(cov_estimated_1) > 0))
print("Estimated Covariance Matrix 2 is positive definite:", np.all(np.linalg.eigvals(cov_estimated_2) > 0))


Covariance Matrix 1 is positive definite: True
Covariance Matrix 2 is positive definite: True
Estimated Covariance Matrix 1:
 [[2.00694718 0.76664407 0.37331473]
 [0.76664407 0.63101849 0.32240379]
 [0.37331473 0.32240379 0.36941065]]
Estimated Covariance Matrix 2:
 [[ 1.28445129 -0.40781698  0.35305686]
 [-0.40781698  0.87535231 -0.13011552]
 [ 0.35305686 -0.13011552  1.01380803]]
Estimated Covariance Matrix 1 is positive definite: True
Estimated Covariance Matrix 2 is positive definite: True


In [29]:
import numpy as np
import mne

# Simulation parameters
channels = 3 # Number of EEG channels
sfreq = 256.0 # Sampling frequency in Hertz
duration = 10.0 # Signal duration in Seconds
change = 5.0 # Time of amplitude change
f = 10.0 + 2.0 * np.random.rand(channels) # Frequencies
phi = 2.0 * np.pi * np.random.rand(channels) # Phases
A1 = 2.0e-6 # Amplitude during first part
A2 = 0.0e-6 # Amplitude during second part
sd = 2.0e-6 # Standard deviation of Gaussian noise

# Create dummy metadata
info = mne.create_info([f'EEG{n:03}' for n in range(1, channels + 1)],
                       ch_types=['eeg'] * channels, sfreq=sfreq)
# Create dummy signals
samples = int(sfreq * duration)
t = np.linspace(0, duration, samples, endpoint=False)
A = A1 + (t > change).astype(float) * (A2 - A1)
data = A * np.cos(np.outer(f, t) + np.outer(phi, np.ones(samples)))
data += np.random.normal(0, sd, size=(channels, samples))
# Create MNE raw object
raw = mne.io.RawArray(data, info)
mne.export.export_raw('dummy.edf', raw, fmt='edf', overwrite=True)

Creating RawArray with float64 data, n_channels=3, n_times=2560
    Range : 0 ... 2559 =      0.000 ...     9.996 secs
Ready.
Overwriting existing file.


In [33]:
raw.get_data()

-2.769297373353031e-07

: 